In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from nbdev import *
%nbdev_default_export base
%nbdev_default_class_level 3

Cells will be exported to geomechy.base,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_export
from geomechy.core import MaterialManager

ok


# Base
> all the base classes

## Base Properties

In [ ]:
%nbdev_export
class Properties:
    
    def __init__ (self, props={}):

        for key in props.keys():
            setattr(self, key, props[key])
    
    def __str__ (self):
        att_prop  = ''
        
        for att in dir(self):
            if att.startswith('__') or att.startswith('store'):
                continue
                
            att_prop += 'Attribute: ' + att + '\n'
            att_prop += str(getattr(self,att)) + '\n'

        return att_prop
    
    def __iter__ ( self ):
        propsList = []
        
        for att in dir(self):
            if att.startswith('__') or att.startswith('store'):
                continue
            propsList.append((att, getattr(self,att)))

        return iter(propsList)
    
    def store (self, key, val):
        setattr(self, key, val)

## Base Material

In [ ]:
%nbdev_export
class BaseMaterial:
    
    def __init__(self, props):
        for name,val in props:
            setattr(self, name, val)

        self.initHistory={}
        self.current = []
        self.iIter = -1

    def setIter(self, iIter):
        self.iIter = iIter

    def setHistoryParameter(self, name, val):
        if self.iIter == -1:
            self.initHistory[name]=val
            return

        if len(self.current) == self.iIter:
            self.current.append(self.initHistory.copy())

        self.current[self.iIter][name] = val
        return 

    def getHistoryParameter(self, name):
        if len(self.history) == 0:
            return self.initHistory[name]
        else:
            return self.history[self.iIter][name]
    
    def commitHistory(self):
        self.history = []
        for h in self.current:
            self.history.append(h)

## Base Element

In [ ]:
%nbdev_export
class BaseElement(list):
    
    dofTypes = []
    
    def __init__ (self, nodes, props):
        list.__init__(self, nodes)
        self.history = {}
        self.current = {}

        for name,val in props:
            if name is "material":
                self.matProps = val
                self.mat = MaterialManager(self.matProps)
            else:
                setattr(self, name, val)
                
    def dofCount(self):
        return len(self)*len(self.dofTypes)
    
    def getNodes(self):
        return self
    
    def getType (self):
        return self.elemType
    
    def setHistoryParameter(self, name, val):
        self.current[name] = val
        
    def getHistoryParameter(self, name):
        return self.history[name]
    
    def commitHistory(self):
        self.history = self.current.copy()
        self.current = {}
        
        if hasattr(self, "mat"):
            self.mat.commitHistory()

ok


## Base Dict

In [ ]:
%nbdev_export
class ItemDict(dict):
    
    def add (self, Id, item):
        self[Id] = item
        
    def get(self, Ids):
        return (self[Ids] if isinstance(Ids,int) else [self[Id] for Id in Ids] 
                if isinstance(Ids,list) else "NAN")

    def getId(self, Ids):
        return (list(self.keys())[Ids] if isinstance(Ids,int) else [list(self.keys())[Id] for Id in Ids] 
                if isinstance(Ids,list) else "NaN" )

In [ ]:
%nbdev_hide
notebook2script()

Converted 00_core.ipynb.
Converted 01_physics.ipynb.
Converted 02_materials.ipynb.
Converted 09_base.ipynb.
Converted 10_utils.ipynb.
Converted index.ipynb.
